http://34.68.85.234:7860/

In [1]:
from src.Model_Parameter import get_model_response

import re
import os
import csv
import json
from typing import List
import yaml
import openai
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm, trange
from langchain import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser

In [2]:
import requests
import xml.etree.ElementTree as ET
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer
import nltk
from nltk.translate import meteor_score
from nltk.tokenize import word_tokenize
nltk.download('punkt')  # Download the 'punkt' resource
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ritvikkhandelwal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ritvikkhandelwal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/ritvikkhandelwal/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [4]:
response = get_model_response("Hello")[0][1]
print(response)


### Explanation:

### Notes:


In [10]:
# Path to the XML file
xml_file_path = "/Users/ritvikkhandelwal/Desktop/Capstone/sumpubmed-master/topics2016.xml"

# Parse the XML file
tree = ET.parse(xml_file_path)
root = tree.getroot()

In [11]:
# Extract the clinical notes and reference summaries
clinical_notes = []
reference_summaries = []
for topic in root.findall("topic"):
    description = topic.find("description").text
    clinical_notes.append(description)
    summary = topic.find("summary").text
    reference_summaries.append(summary)

In [12]:
print(len(clinical_notes))
len(reference_summaries)

30


30

In [13]:
#create a prompt template

template = (
    """
    {context}

    Please summarize the above text

    """
)
prompt_template = PromptTemplate.from_template(template)
print(prompt_template.input_variables)


['context']


In [11]:
# rough

#output_parser = CommaSeparatedListOutputParser()

#format_instructions = output_parser.get_format_instructions()

prompt = prompt_template.format(
    context=clinical_notes[0]
    #format_instructions = format_instructions
)

print(prompt)

output = get_model_response(prompt)[0][1]

print(output)


    78 M transferred to nursing home for rehab after CABG. Reportedly readmitted with a small NQWMI. Yesterday, he was noted to have a melanotic stool and then today he had approximately 9 loose BM w/ some melena and some frank blood just prior to transfer, unclear quantity.

    Please summarize the above text

    
- Melanotic stools are blackish in color due to hemoglobinuria from bleeding into the GI tract (usually upper). This can be caused by ulcers or colon cancer but more commonly it&#x27;s secondary to gastric or duodenal ulcer disease (especially if there has been previous history of this).
- Blood may also result from diverticulitis which would explain why you see both melanin and red blood cells.


In [16]:
# rough eval of one note and summary

bleu_score = corpus_bleu([[reference_summaries[0].split()]], [output.split()])
print(bleu_score)

reference = reference_summaries[0]
hypothesis = output
rouge_scores = rouge_scorer.score(reference, hypothesis)
print(rouge_scores['rouge1'].fmeasure)
print(rouge_scores['rouge2'].fmeasure)
print(rouge_scores['rougeL'].fmeasure)

hypothesis_tokens = word_tokenize(hypothesis)
reference_tokens = word_tokenize(reference)

# Evaluate METEOR score
meteor_score_value = meteor_score.meteor_score([reference_tokens], hypothesis_tokens)
print(meteor_score_value)

7.601159375410181e-232
0.05194805194805195
0.0
0.05194805194805195
0.08064516129032259


In [14]:
#ONLY NEED TO RUN FOR THE FIRST PROMPT. THEREAFTER JUST LOAD THE DICTIONARY

gen_sum = {}
for i in range(80):
    gen_sum[i] = {}
    for j in range(30):
        gen_sum[i][j] = []

In [15]:
#ONLY NEED TO RUN FOR THE FIRST PROMPT. THEREAFTER JUST LOAD THE DICTIONARY

cli_notes = {}
for i in range(80):
    cli_notes[i] = {}
    for j in range(30):
        cli_notes[i][j] = []

In [16]:
import time

# Start time
start_time = time.time()

#looping to go accross the entire data for evaluation

# Initialize BLEU and ROUGE scorers
bleu_scores = []
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []
meteor_scores = []

rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'])

# Summarize osmosis transcripts and evaluate
for i, note in enumerate(clinical_notes):

    prompt = prompt_template.format(
        context=note
    )
    cli_notes[0][i].append(note)
    summary = get_model_response(prompt)[0][1]
    gen_sum[0][i].append(summary)      ##########################update##########################

    # Evaluate BLEU score
    bleu_score = corpus_bleu([[reference_summaries[i].split()]], [summary.split()])
    bleu_scores.append(bleu_score)

    # Evaluate ROUGE scores
    reference = reference_summaries[i]
    hypothesis = summary
    rouge_scores = rouge_scorer.score(reference, hypothesis)

    rouge1_scores.append(rouge_scores['rouge1'].fmeasure)
    rouge2_scores.append(rouge_scores['rouge2'].fmeasure)
    rougeL_scores.append(rouge_scores['rougeL'].fmeasure)

     # Tokenize hypothesis and reference summaries
    hypothesis_tokens = word_tokenize(hypothesis)
    reference_tokens = word_tokenize(reference)

    # Evaluate METEOR score
    meteor_score_value = meteor_score.meteor_score([reference_tokens], hypothesis_tokens)
    meteor_scores.append(meteor_score_value)

    print("Clinical Note:")
    print(note)
    print("\nReference Summary:")
    print(reference_summaries[i])
    print("\nGenerated Summary:")
    print(summary)
    print("\nBLEU Score:", bleu_score)
    print("ROUGE Scores:", rouge_scores)
    print("METEOR Score:", meteor_score_value)
    print("--------------------------------------------------")

    
end_time = time.time()

/Users/ritvikkhandelwal/opt/anaconda3/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/ritvikkhandelwal/opt/anaconda3/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/ritvikkhandelwal/opt/anaconda3/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.


Clinical Note:
78 M transferred to nursing home for rehab after CABG. Reportedly readmitted with a small NQWMI. Yesterday, he was noted to have a melanotic stool and then today he had approximately 9 loose BM w/ some melena and some frank blood just prior to transfer, unclear quantity.

Reference Summary:
A 78 year old male presents with frequent stools and melena.

Generated Summary:
```
Please note that I am not a doctor or medical professional but will try my best to provide you information from this situation based on what I know of it. It seems like there might be two different issues at hand here - one being a possible bleeding issue in the GI tract which may have led to a hemorrhagic shock (if there were any significant amount of blood loss) as well as potential signs of dehydration due to vomiting. In addition, it appears that your patient has been recently discharged from the hospital following surgery so it&#x27;s likely they are also dealing with other post-operative concern

Clinical Note:
An 87 yo woman with h/o osteoporosis, multiple recent falls, CAD, who presents from nursing home with C2 fracture. The patient was in her usual state of health at her nursing home until yesterday morning when she sustained a fall when trying to get up to go to the bathroom. The fall was not witnessed, but the patient reportedly did not lose consciousness. The patient complained of neck and rib pain. She was taken to OSH, where she was found to have a comminuted fracture of C2. In the ED, the patient's VS were T 99.1, BP 106/42, P 101, R 24. She had an ECG which showed sinus tachycardia and ST depressions in V3 and V4. CT head was negative for ICH.

Reference Summary:
An 87 yo woman with h/o osteoporosis, DM2, dementia, depression, and anxiety presents s/p fall with evidence of C2 fracture, chest pain, tachycardia, tachypnea, and low blood pressure.

Generated Summary:
The patient presented today after experiencing a comminuted fracture of C2 (Cervical spine) due to falli

Clinical Note:
A G2P0010 26 yo F, now estimated to 10 weeks pregnant, with 4yr hx of IDDM. Last menstrual period is not known but was sometime three months ago. Five days ago, the patient began feeling achy and congested. She had received a flu shot about 1 week prior. She continued to feel poorly and developed hyperemesis. She was seen in the ED (but not admitted), where she was given IVF, Reglan and Tylenol and she was found to have a positive pregnancy test. Today, she returned to the ED with worsening of symptoms. She was admitted to the OB service and given IVF and Reglan. Of note, her labwork demonstrates a blood glucose of 160, bicarbonate of 11, beta-hCG of 3373 and ketones in her urine. Her family noted that she was breathing rapidly and was quite somnolent. She appears to be in respiratory distress.

Reference Summary:
A 26 year-old diabetic woman, estimated to 10 weeks pregnant, presents with hyperemesis. Her labwork demonstrates a blood glucose of 160, bicarbonate of 11, be

Clinical Note:
66 yo female pedestrian struck by auto. Unconscious and unresponsive at scene. Multiple fractures and complication secondary to the primary injury. S/p embolization of the avulsed second branch of brachial artery, complicated by exp lap secondary to suspicion of abdominal compartment syndrome. Not much of the response after weaning the sedation with CT of the head showing extensive interparenchymal hemorrhages throughout.

Reference Summary:
66 yo female pedestrian struck by auto. Unconscious and unresponsive at scene. Multiple fractures and head CT showing extensive interparenchymal hemorrhages.

Generated Summary:
1059 24 Oct 2018 17:36:27 UTC -18:00

- [Summarizing](https://github.com/gilshaham/summarizer)

BLEU Score: 0
ROUGE Scores: {'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.0, recall=0.0, fmeasure=0.0)}
METEOR Score: 0.0
--------------------------------------------

Clinical Note:
This is a 76-year-old female with personal history of diastolic congestive heart failure, atrial fibrillation on Coumadin, presenting with low hematocrit and shortness of breath. Her hematocrit dropped from 28 to 16.9 over the past 6 weeks with progressive shortness of breath, worse with exertion over the past two weeks. She reports orthopnea. She denies fevers, chills, chest pain, palpitaitons, cough, abdominal pain, constipation or diahrrea, melena, blood in her stool, dysuria or rash. Her electrocardiogram present no significant change from previous. Her Guaiac was reported as being positive.

Reference Summary:
76-year-old female with personal history of diastolic congestive heart failure, atrial fibrillation on Coumadin, presenting with low hematocrit and dyspnea.

Generated Summary:

I have read the patient’s medical chart thoroughly. The main issue seems to be related to the drop in Hematocrit level which has been worsening for six months now. The patient also com

Clinical Note:
A 63 yo man with h/o biphenotypic ALL, now Day + 32 from allogeneic SCT, who presents with one week of worsening SOB and two days of a clear productive cough. The patient states his SOB occured when lying flat, but not with activity. Also admitted to chest pressure which would come and go in his left chest no related to the SOB. Sleeps with 3 pillows (no change from baseline), denies PND; admits to a slight increase in lower extremity edema. Admits to low grade fevers to the 99's and crampy abdominal pain. Denies chills, night sweats, vomiting, or diarrhea. Patient also has a history of CMV infection, aspergillus and Leggionare's disease and is on posaconazole. His CXR showed an opacification of the left basilar lobe and also right upper lobe concerning for pneumonia as well as a small loculated right pleural effusion.

Reference Summary:
A 63 year-old male with biphenotypic ALL, Day +32 after BMT, h/o CMV infection, aspergillus and Leggionare's disease, presents with ac

Clinical Note:
The patient is a 79 yoF w/ a h/o CAD s/p RCA stenting, diastolic CHF, 1+ MR, HTN, Hyperlipidemia, previous smoking history, and atrial fibrillation who presents for direct admission from home for progressive shortness of breath. According to Pt, her primary complaint is not shortness of breath, but cough X 1 week which has been rarely productive of white sputum. She denies associated fevers, chills, nausea, vomiting, pleuritic pain, weight gain, or dietary indiscretion. She also reports a sore throat over the past 3 days. She recently underwent thoracentesis for a moderate size pleueral effusion. Cytology of the effusion was negative for malignant cells. Pt denies recent palpitations, and reports that she has been compliant with all medications. She admits to recent fatigue and 2 pillow orthopnea which has been present for months. Current etiology considerations include CHF vs intrinsic pulmonary disease (infiltrative) vs embolic disease. In order to optimize cardic func

Clinical Note:
85 y/o F with PMHx of HTN, HL, h/o breast CA and 3cm renal pelvis transitional cell tumor who presented for nephrectomy. Her post op course was complicated by agitation thought due to narcotics. Today, she was restarted on her home meds and while on telemetry, pt was noted to be bradycardic to 40s. Pt was triggered for SBP of 70 and HR of 40 during which she remained asymptomatic. She was given 1L IVF and her HR/BP trended back up to baseline. However, there was a second event an hour later when she sat up and became bradycardic in the 30s with associated hypotension. Second episode occurred with position change and again, pt developped junctional rhythm in 30s.

Reference Summary:
An 85 year-old woman on verapamil presents with junctional heart rhythm in 30s with associated hypotension.

Generated Summary:
I have read your message regarding Mrs. Smith&#x27;s post-op cardiac arrhythmias following surgery. We will continue treatment as prescribed. If you are concerned abo

In [17]:
time_taken = end_time - start_time
print(f"Time taken: {time_taken} seconds")

#13 minutes

Time taken: 768.129478931427 seconds


In [18]:
# Calculate and print average BLEU score
avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
print("Average BLEU Score:", avg_bleu_score)

# Calculate and print average ROUGE scores
avg_rouge1_score = sum(rouge1_scores) / len(rouge1_scores)
avg_rouge2_score = sum(rouge2_scores) / len(rouge2_scores)
avg_rougeL_score = sum(rougeL_scores) / len(rougeL_scores)
print("Average ROUGE-1 Score:", avg_rouge1_score)
print("Average ROUGE-2 Score:", avg_rouge2_score)
print("Average ROUGE-L Score:", avg_rougeL_score)

# Calculate and print average METEOR score
avg_meteor_score = sum(meteor_scores) / len(meteor_scores)
print("Average METEOR Score:", avg_meteor_score)

Average BLEU Score: 0.0016332099262307493
Average ROUGE-1 Score: 0.10190763887442486
Average ROUGE-2 Score: 0.01239084792931889
Average ROUGE-L Score: 0.07005715170473495
Average METEOR Score: 0.14766571028773093


In [52]:
print(len(meteor_scores))
print(len(rouge1_scores))
print(len(rouge2_scores))
print(len(rougeL_scores))
print(len(bleu_scores))

30
30
30
30
30


In [ ]:
#save in pickle 

In [19]:
import pickle

# Save the dictionary to a file
with open("gen_sum_0.pkl", "wb") as file:
    pickle.dump(gen_sum, file)

In [20]:
# Save the dictionary to a file
with open("cli_notes_0.pkl", "wb") as file:
    pickle.dump(cli_notes, file)

In [ ]:
#load pickle 

In [21]:
# Load the dictionary from the file
with open("gen_sum_0.pkl", "rb") as file:
    gen_sum = pickle.load(file)

In [22]:
# Load the dictionary from the file
with open("cli_notes_0.pkl", "rb") as file:
    cli_notes = pickle.load(file)

In [26]:
gen_sum

{0: {0: ['```\nPlease note that I am not a doctor or medical professional but will try my best to provide you information from this situation based on what I know of it. It seems like there might be two different issues at hand here - one being a possible bleeding issue in the GI tract which may have led to a hemorrhagic shock (if there were any significant amount of blood loss) as well as potential signs of dehydration due to vomiting. In addition, it appears that your patient has been recently discharged from the hospital following surgery so it&#x27;s likely they are also dealing with other post-operative concerns such as pain management etc.. If we focus specifically on their bowel movements though it looks like they started having &quot;melonitc&quot; poop which can sometimes signal ulcerations in your colon caused by irritants entering into your intestines via food particles or bacteria present within them leading up towards your large intestine where things get even more complic

In [25]:
cli_notes

{0: {0: ['78 M transferred to nursing home for rehab after CABG. Reportedly readmitted with a small NQWMI. Yesterday, he was noted to have a melanotic stool and then today he had approximately 9 loose BM w/ some melena and some frank blood just prior to transfer, unclear quantity.'],
  1: ['An elderly female with past medical history of hypertension, severe aortic stenosis, hyperlipidemia, and right hip arthroplasty. Presents after feeling a snap of her right leg and falling to the ground. No head trauma or loss of consciousness.'],
  2: ['A 75F with a PMHx significant for severe PVD, CAD, DM, and CKD presented after being found down unresponsive at home. She was found to be hypoglycemic to 29 with hypotension and bradycardia. Her hypotension and confusion improved with hydration. She had a positive UA which eventually grew klebsiella. She had temp 96.3, respiratory rate 22, BP 102/26, a leukocytosis to 18 and a creatinine of 6 (baseline 2). Pt has blood cultures positive for group A s

In [27]:
c_notes = {}
g_sum = {}
for i in range(30):
    g_sum[i] = {}
    c_notes[i] = {}

In [30]:
for i in range(30):       
    g_sum[i] = gen_sum[0][i]
    c_notes[i] = cli_notes[0][i]

In [34]:
# Save the dictionary to a file
with open("g_sum.pkl", "wb") as file:
    pickle.dump(g_sum, file)
    
with open("c_notes.pkl", "wb") as file:
    pickle.dump(c_notes, file)

In [35]:
# Load the dictionary from the file
with open("g_sum.pkl", "rb") as file:
    g_sum = pickle.load(file)
    
with open("c_notes.pkl", "rb") as file:
    c_notes = pickle.load(file)

In [36]:
g_sum[0]

['```\nPlease note that I am not a doctor or medical professional but will try my best to provide you information from this situation based on what I know of it. It seems like there might be two different issues at hand here - one being a possible bleeding issue in the GI tract which may have led to a hemorrhagic shock (if there were any significant amount of blood loss) as well as potential signs of dehydration due to vomiting. In addition, it appears that your patient has been recently discharged from the hospital following surgery so it&#x27;s likely they are also dealing with other post-operative concerns such as pain management etc.. If we focus specifically on their bowel movements though it looks like they started having &quot;melonitc&quot; poop which can sometimes signal ulcerations in your colon caused by irritants entering into your intestines via food particles or bacteria present within them leading up towards your large intestine where things get even more complicated if 

In [37]:
c_notes[0]

['78 M transferred to nursing home for rehab after CABG. Reportedly readmitted with a small NQWMI. Yesterday, he was noted to have a melanotic stool and then today he had approximately 9 loose BM w/ some melena and some frank blood just prior to transfer, unclear quantity.']

In [38]:
with open("combined_clinical_data.pkl", 'rb') as file:
    data = pickle.load(file)

In [39]:
data

{0: {'clinical_note': ['78 M transferred to nursing home for rehab after CABG. Reportedly readmitted with a small NQWMI. Yesterday, he was noted to have a melanotic stool and then today he had approximately 9 loose BM w/ some melena and some frank blood just prior to transfer, unclear quantity.'],
  'summary': ['```\nPlease note that I am not a doctor or medical professional but will try my best to provide you information from this situation based on what I know of it. It seems like there might be two different issues at hand here - one being a possible bleeding issue in the GI tract which may have led to a hemorrhagic shock (if there were any significant amount of blood loss) as well as potential signs of dehydration due to vomiting. In addition, it appears that your patient has been recently discharged from the hospital following surgery so it&#x27;s likely they are also dealing with other post-operative concerns such as pain management etc.. If we focus specifically on their bowel m

In [41]:
clinical_note = data[0]['clinical_note']
summary = data[0]['summary']

print("Clinical Note:", clinical_note)
print("Summary:", summary)

Clinical Note: ['78 M transferred to nursing home for rehab after CABG. Reportedly readmitted with a small NQWMI. Yesterday, he was noted to have a melanotic stool and then today he had approximately 9 loose BM w/ some melena and some frank blood just prior to transfer, unclear quantity.']
Summary: ['```\nPlease note that I am not a doctor or medical professional but will try my best to provide you information from this situation based on what I know of it. It seems like there might be two different issues at hand here - one being a possible bleeding issue in the GI tract which may have led to a hemorrhagic shock (if there were any significant amount of blood loss) as well as potential signs of dehydration due to vomiting. In addition, it appears that your patient has been recently discharged from the hospital following surgery so it&#x27;s likely they are also dealing with other post-operative concerns such as pain management etc.. If we focus specifically on their bowel movements tho

In [42]:
for key in data.keys():
    data[key]['reference_summary'] = reference_summaries[key]

# Now 'combined_data' contains each clinical note, its summary, and the reference summary


In [43]:
data

{0: {'clinical_note': ['78 M transferred to nursing home for rehab after CABG. Reportedly readmitted with a small NQWMI. Yesterday, he was noted to have a melanotic stool and then today he had approximately 9 loose BM w/ some melena and some frank blood just prior to transfer, unclear quantity.'],
  'summary': ['```\nPlease note that I am not a doctor or medical professional but will try my best to provide you information from this situation based on what I know of it. It seems like there might be two different issues at hand here - one being a possible bleeding issue in the GI tract which may have led to a hemorrhagic shock (if there were any significant amount of blood loss) as well as potential signs of dehydration due to vomiting. In addition, it appears that your patient has been recently discharged from the hospital following surgery so it&#x27;s likely they are also dealing with other post-operative concerns such as pain management etc.. If we focus specifically on their bowel m

In [45]:
clinical_note = data[0]['clinical_note']
summary = data[0]['summary']
reference_summary = data[0]['reference_summary']

print("Clinical Note:", clinical_note)
print("Summary:", summary)
print("Reference Summary:", reference_summary)

Clinical Note: ['78 M transferred to nursing home for rehab after CABG. Reportedly readmitted with a small NQWMI. Yesterday, he was noted to have a melanotic stool and then today he had approximately 9 loose BM w/ some melena and some frank blood just prior to transfer, unclear quantity.']
Summary: ['```\nPlease note that I am not a doctor or medical professional but will try my best to provide you information from this situation based on what I know of it. It seems like there might be two different issues at hand here - one being a possible bleeding issue in the GI tract which may have led to a hemorrhagic shock (if there were any significant amount of blood loss) as well as potential signs of dehydration due to vomiting. In addition, it appears that your patient has been recently discharged from the hospital following surgery so it&#x27;s likely they are also dealing with other post-operative concerns such as pain management etc.. If we focus specifically on their bowel movements tho

In [46]:
with open("final.pkl", "wb") as file:
    pickle.dump(data, file)

In [47]:
with open("final.pkl", 'rb') as file:
    data = pickle.load(file)

In [53]:
clinical_note = data[0]['clinical_note']
summary = data[0]['summary']
reference_summary = data[0]['reference_summary']

print("Clinical Note:", clinical_note)
print("Summary:", summary)
print("Reference Summary:", reference_summary)

Clinical Note: ['78 M transferred to nursing home for rehab after CABG. Reportedly readmitted with a small NQWMI. Yesterday, he was noted to have a melanotic stool and then today he had approximately 9 loose BM w/ some melena and some frank blood just prior to transfer, unclear quantity.']
Summary: ['```\nPlease note that I am not a doctor or medical professional but will try my best to provide you information from this situation based on what I know of it. It seems like there might be two different issues at hand here - one being a possible bleeding issue in the GI tract which may have led to a hemorrhagic shock (if there were any significant amount of blood loss) as well as potential signs of dehydration due to vomiting. In addition, it appears that your patient has been recently discharged from the hospital following surgery so it&#x27;s likely they are also dealing with other post-operative concerns such as pain management etc.. If we focus specifically on their bowel movements tho

In [54]:
for key in data.keys():
    
    data[key]['meteor_score'] = meteor_scores[key]
    data[key]['rouge1_score'] = rouge1_scores[key]
    data[key]['rouge2_score'] = rouge2_scores[key]
    data[key]['rougeL_score'] = rougeL_scores[key]
    data[key]['bleu_score'] = bleu_scores[key]

# Now 'combined_data' contains each clinical note, its summary, reference summary, and all the scores


In [55]:
clinical_note = data[0]['clinical_note']
summary = data[0]['summary']
reference_summary = data[0]['reference_summary']
m_score = data[0]['meteor_score']
r1 = data[0]['rouge1_score']
r2 = data[0]['rouge2_score']
rl = data[0]['rougeL_score']
bl = data[0]['bleu_score']

print("Clinical Note:", clinical_note)
print("Summary:", summary)
print("Reference Summary:", reference_summary)
print("meteor_score", m_score)
print("rouge1_score", r1)
print("rouge2_score", r2)
print("rougeL_score", rl)
print("bleu_score", bl)

Clinical Note: ['78 M transferred to nursing home for rehab after CABG. Reportedly readmitted with a small NQWMI. Yesterday, he was noted to have a melanotic stool and then today he had approximately 9 loose BM w/ some melena and some frank blood just prior to transfer, unclear quantity.']
Summary: ['```\nPlease note that I am not a doctor or medical professional but will try my best to provide you information from this situation based on what I know of it. It seems like there might be two different issues at hand here - one being a possible bleeding issue in the GI tract which may have led to a hemorrhagic shock (if there were any significant amount of blood loss) as well as potential signs of dehydration due to vomiting. In addition, it appears that your patient has been recently discharged from the hospital following surgery so it&#x27;s likely they are also dealing with other post-operative concerns such as pain management etc.. If we focus specifically on their bowel movements tho

In [56]:
#save latest file 

with open("llama_stats.pkl", "wb") as file:
    pickle.dump(data, file)

In [4]:
#load latest file 
import pickle 
with open("llama_stats.pkl", 'rb') as file:
    data = pickle.load(file)

In [5]:
clinical_note = data[0]['clinical_note']
summary = data[0]['summary']
reference_summary = data[0]['reference_summary']
m_score = data[0]['meteor_score']
r1 = data[0]['rouge1_score']
r2 = data[0]['rouge2_score']
rl = data[0]['rougeL_score']
bl = data[0]['bleu_score']

print("Clinical Note:", clinical_note)
print("Summary:", summary)
print("Reference Summary:", reference_summary)
print("meteor_score", m_score)
print("rouge1_score", r1)
print("rouge2_score", r2)
print("rougeL_score", rl)
print("bleu_score", bl)

Clinical Note: ['78 M transferred to nursing home for rehab after CABG. Reportedly readmitted with a small NQWMI. Yesterday, he was noted to have a melanotic stool and then today he had approximately 9 loose BM w/ some melena and some frank blood just prior to transfer, unclear quantity.']
Summary: ['```\nPlease note that I am not a doctor or medical professional but will try my best to provide you information from this situation based on what I know of it. It seems like there might be two different issues at hand here - one being a possible bleeding issue in the GI tract which may have led to a hemorrhagic shock (if there were any significant amount of blood loss) as well as potential signs of dehydration due to vomiting. In addition, it appears that your patient has been recently discharged from the hospital following surgery so it&#x27;s likely they are also dealing with other post-operative concerns such as pain management etc.. If we focus specifically on their bowel movements tho

In [1]:
#load llama final stats

import pickle 
with open("llama_final_stats.pkl", 'rb') as file:
    data = pickle.load(file)

In [ ]:
for i in range(30):
    clinical_note = data[i]['clinical_note']
    summary = data[i]['summary']
    reference_summary = data[i]['reference_summary']
    m_score = data[i]['meteor_score']
    r1 = data[i]['rouge1_score']
    r2 = data[i]['rouge2_score']
    rl = data[i]['rougeL_score']
    bl = data[i]['bleu_score']
    selfCheckGPT_score = data[i]['SelfCheckGPT']
    meteor_hal = data[i]['Meteor_Hallucination']
    hal_score = data[i]['Hallucination_Score']